## Flags and vars

In [ ]:
# MODIFIED
SaveTrendPlots = True
SaveTrendComparisonsPlots = True
SaveTierMaps = True
SaveSlopeMap = True
SaveHistogram = True

YearsToAverage = 10 # years to average when computing end-start diff

## Install packages

In [ ]:
pip install geopandas

## Import standard libraries

In [ ]:
import os
import sys
# append coeqwal packages to path
sys.path.append('./coeqwalpackage')
import datetime as dt
import pandas as pd
import numpy as np
import cqwlutils as cu
import re
import matplotlib.pyplot as plt
import math
import geopandas as gpd
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from pathlib import Path
from typing import Optional
from sklearn.linear_model import LinearRegression
import matplotlib.colors as mcolors
from matplotlib.ticker import FixedLocator, FixedFormatter

%matplotlib inline



## Import custom modules

In [ ]:
# Import custom modules - NEED WINDOWS OS (NOTE: I had to run this twice, must check why this happens!)
from coeqwalpackage.DataExtraction import *
from coeqwalpackage.metrics import *


# Specify baseline scenario and other constants

In [ ]:
# MODIFIED
baseline_scenario = "s0002"
end_year_override = {"s0006", "s0007", "s0008", "s0009", "s0010"} #scenarios that end before others
drop_threshold = 1000
start_year = 1960
start_date = pd.Timestamp("1990-01-01")
end_date = pd.Timestamp("2000-12-31")
slope_start_date="1992-09-30"
window_start = pd.Timestamp("1973-10-31") # GW simulation start and end
window_end   = pd.Timestamp("2015-09-30")
monthly_ft_filename = "GroundWater_Levels_Monthly.csv"
annual_ft_filename = "GroundWater_Levels_Annual.csv"
monthly_af_filename = "GroundWater_Volumes_Monthly.csv"
annual_af_filename = "GroundWater_Volumes_Annual.csv"
monthly_volper_filename = "GroundWater_Levels_BaselinePercent_Monthly.csv"
annual_volper_filename = "GroundWater_Levels_BaselinePercent_Annual.csv"
monthly_levper_filename = "GroundWater_Levels_BaselinePercent_Monthly.csv"
annual_levper_filename = "GroundWater_Levels_BaselinePercent_Annual.csv"
monthly_volpercv_filename = "GroundWater_Levels_BaselinePercentCV_Monthly.csv"
annual_volpercv_filename = "GroundWater_Levels_BaselinePercentCV_Annual.csv"
monthly_levpercv_filename = "GroundWater_Levels_BaselinePercentCV_Monthly.csv"
annual_levpercv_filename = "GroundWater_Levels_BaselinePercentCV_Annual.csv"
monthly_ft_cv_filename = "GroundWater_Levels_CV_monthly.csv"
annual_ft_cv_filename = "GroundWater_Levels_CV_annual.csv"
monthly_af_cv_filename = "GroundWater_Volumes_CV_monthly.csv"
annual_af_cv_filename = "GroundWater_Volumes_CV_annual.csv"
trend_filename = "GroundWater_Trends_ft_per_month.csv"
diff_filename = "GroundWater_AvgEndStartDiff_ft.csv"
GWregionIndex_filename = "CalSim3GWregionIndex.wresl"
WBAIndex_filename = "CalSim3_WBA.csv"
WBAStorage_filename = "20250908draft_C2VSim_73-15_WBA_Storage.csv"
GWwreslmapping_filename = "groundwater_wresl_mapping.csv"
tier_filename = "GroundWater_Tiers.csv"
shapefile_filename = "./shapefiles (1)/i12_CalSim3Model_WaterBudgetAreas_20221021.shp"


## Create directories

In [ ]:
# MODIFIED
def find_repo_root(marker_dirs=("CalSim3_Model_Runs", "coeqwal")) -> Path:
    """
    Walk upward from the script (or notebook CWD) until we find a directory
    that contains the expected project folders (e.g., 'CalSim3_Model_Runs' and 'coeqwal').
    """
    # 1) Optional env override
    env_root = os.environ.get("DSP_REPO_ROOT")
    if env_root:
        root = Path(env_root).expanduser().resolve()
        if all((root / m).exists() for m in marker_dirs):
            return root

    # 2) Start from script dir if available; else notebook working dir
    try:
        start = Path(__file__).resolve().parent
    except NameError:
        start = Path.cwd().resolve()

    # 3) Walk up until markers are found
    for parent in [start] + list(start.parents):
        if all((parent / m).exists() for m in marker_dirs):
            return parent

    raise FileNotFoundError(
        "Could not locate repo root containing: "
        + ", ".join(marker_dirs)
        + ". Set DSP_REPO_ROOT env var or run from inside the repo."
    )

repo_root = find_repo_root()

output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics" / "Tiered_Outcome_Measures" / "Groundwater"
output_dir.mkdir(parents=True, exist_ok=True)
trends_output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics" / "Tiered_Outcome_Measures" / "Groundwater" / "TrendPlots"
trends_output_dir.mkdir(parents=True, exist_ok=True)
trends_comparisons_output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics" / "Tiered_Outcome_Measures" / "Groundwater" / "TrendComparisonPlots"
trends_comparisons_output_dir.mkdir(parents=True, exist_ok=True)
data_output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics" / "Tiered_Outcome_Measures" / "Groundwater" / "Data"
data_output_dir.mkdir(parents=True, exist_ok=True)
tiers_output_dir = repo_root / "CalSim3_Model_Runs" / "Scenarios" / "Performance_Metrics" / "Tiered_Outcome_Measures" / "Groundwater" / "TierMaps"
tiers_output_dir.mkdir(parents=True, exist_ok=True)


## Set paths

In [ ]:
base_dir = os.path.abspath(".")
wba_storage_csv_path = os.path.join(base_dir, WBAStorage_filename)
output_mappingcsv_path = os.path.join(base_dir, GWwreslmapping_filename)
wresl_path = os.path.join(base_dir, GWregionIndex_filename)
wba_csv_path = os.path.join(base_dir, WBAIndex_filename)
tier_output_path = os.path.join(data_output_dir, tier_filename)
shapefile_path = os.path.join(base_dir, shapefile_filename)

## Define contol file name

In [ ]:
CtrlFile = 'CalSim3GroundWaterDataExtractionInitFile_v1.xlsx'
CtrlTab = 'Init'

## Read from control file

In [ ]:
ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax = cu.read_init_file(CtrlFile, CtrlTab)

In [ ]:
print([ScenarioListFile, ScenarioListTab, ScenarioListPath, GW1DssNamesOutPath, GW2DssNamesOutPath, ScenarioIndicesOutPath, DssDirsOutPath, VarListPath, VarListFile, VarListTab, VarOutPath, DataOutPath, ConvertDataOutPath, ExtractionSubPath, DemandDeliverySubPath, ModelSubPath, GroupDataDirPath, ScenarioDir, GW1DssMin, GW1DssMax, GW2DssMin, GW2DssMax, NameMin, NameMax, DirMin, DirMax, IndexMin, IndexMax, StartMin, StartMax, EndMin, EndMax, VarMin, VarMax, DemandFilePath, DemandFileName, DemandFileTab, DemMin, DemMax, InflowOutSubPath, InflowFilePath, InflowFileName, InflowFileTab, InflowMin, InflowMax])


## Check for output directory and create if necessary (not necessary)

In [ ]:
# check if output directory exists
if not os.path.exists(GroupDataDirPath):
    # print warning
    print("Warning: directory " + GroupDataDirPath + " does not exists and will be created")
    
    # Create the directory
    os.makedirs(GroupDataDirPath)


## Define Nan Values

In [ ]:
# NaN values as defined by CalSim3
Nan1 = -901
Nan2 = -902

## Read indeces, dss names, directory names, start and end dates, time range (not necessary)

In [ ]:
gw1dsshdr, gw1dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW1DssMin, GW1DssMax, hdr=True)
gw1dss_names = []
for i in range(len(gw1dssname)):
    gw1dss_names.append(gw1dssname[i][0])
gw1dss_names

In [ ]:
gw2dsshdr, gw2dssname = cu.read_from_excel(ScenarioListPath, ScenarioListTab, GW2DssMin, GW2DssMax, hdr=True)
gw2dss_names = []
for i in range(len(gw2dssname)):
    gw2dss_names.append(gw2dssname[i][0])
gw2dss_names

In [ ]:
indexhdr, index_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, IndexMin, IndexMax, hdr=True)
index_names = []
for i in range(len(index_name)):
    index_names.append(index_name[i][0])
index_names

In [ ]:
studyhdr, study_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, NameMin, NameMax, hdr=True)
study_names = []
for i in range(len(study_name)):
    study_names.append(study_name[i][0])
study_names

In [ ]:
dirhdr, dir_name = cu.read_from_excel(ScenarioListPath, ScenarioListTab, DirMin, DirMax, hdr=True)
dir_names = []
for i in range(len(dir_name)):
    dir_names.append(dir_name[i][0])
dir_names

In [ ]:
starthdr, start_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, StartMin, StartMax, hdr=True)
start_dates = []
for i in range(len(start_date)):
    start_dates.append(start_date[i][0])
datetime_start_dates = pd.to_datetime(start_dates)
# turns out that dss reading library wands a dt datetime, not pd datetime
dt_datetime_start_dates = [dt.to_pydatetime() for dt in datetime_start_dates]


In [ ]:
endhdr, end_date = cu.read_from_excel(ScenarioListPath, ScenarioListTab, EndMin, EndMax, hdr=True)
end_dates = []
for i in range(len(end_date)):
    end_dates.append(end_date[i][0])
# turns out that dss reading library wands a dt datetime, not pd datetime
datetime_end_dates = pd.to_datetime(end_dates)
dt_datetime_end_dates = [dt.to_pydatetime() for dt in datetime_end_dates]


In [ ]:
min_datetime = min(dt_datetime_start_dates)
print('Min time: ')
print(min_datetime)
max_datetime = max(dt_datetime_end_dates)
print('Max time: ')
print(max_datetime)


## Read variables list (not necessary)

In [ ]:
# get vars
hdr, vars = cu.read_from_excel(VarListPath, VarListTab,VarMin,VarMax,hdr=True)
gw1var_df = pd.DataFrame(data=vars, columns=hdr)
gw1var_df

In [ ]:
print(gw1var_df.head(20))   # show first 20 rows
print(gw1var_df.columns)    # see what metadata is included
print(gw1var_df.shape)      # rows × cols


## Read the compund data from CSV to df

In [ ]:
# read the dataframe from CSV
print('Reading ' + DataOutPath)
gw1_df, gw1dss_names = read_in_df(DataOutPath,GW1DssNamesOutPath)

In [ ]:
print("gw1dss_names:")
gw1dss_names

In [ ]:
print("gw1_df:")
gw1_df

## Drop the LT:E999 columns

In [ ]:
mask = ~gw1_df.columns.to_frame().apply(lambda col: col.astype(str).str.contains('LT:E999')).any(axis=1)
gw1_df = gw1_df.loc[:, mask.values]

In [ ]:
print("new gw1_df:")
gw1_df

## Add water year column to df

In [ ]:
def add_water_year_column(df):
    df_copy = df.copy().sort_index()
    df_copy['Date'] = pd.to_datetime(df_copy.index)
    df_copy.loc[:, 'Year'] = df_copy['Date'].dt.year
    df_copy.loc[:, 'Month'] = df_copy['Date'].dt.month
    df_copy.loc[:, 'WaterYear'] = np.where(df_copy['Month'] >= 10, df_copy['Year'] + 1, df_copy['Year'])
    return df_copy.drop(["Date", "Year", "Month"], axis=1)

In [ ]:
gw1_df = add_water_year_column(gw1_df)

In [ ]:
print("gw1_df with water year column:")
gw1_df

## End of initialization

In [ ]:
print('Done Initializing!')

## Map SR to WBA

In [ ]:
# MODIFIED
# Read the area data (CSV)
wba_df = pd.read_csv(wba_csv_path)

# Check what columns are present
print("Columns in WBA Area CSV:", wba_df.columns)

# Preview key columns
print(wba_df[['fid', 'GIS_Acres']].head())

# === Step 3: Parse WRESL File to Map SRxx to WBAxx ===

with open(wresl_path, 'r') as f:
    wresl_lines = f.readlines()

# Extract SRxx → WBAxx or DETAW
sr_to_wba_map = {}
for line in wresl_lines:
    # Match standard WBA format: indxWBA_XX = SRYY
    match = re.match(r'\s*indxWBA_(\d+)\s*=\s*(SR\d+)', line)
    if match:
        wba_num, sr_num = match.groups()
        sr_to_wba_map[sr_num] = f'WBA{wba_num}'
    else:
        # Match DETAW format: indxDETAW = SRYY
        match_detaw = re.match(r'\s*indxDETAW\s*=\s*(SR\d+)', line)
        if match_detaw:
            sr_num = match_detaw.group(1)
            sr_to_wba_map[sr_num] = 'DETAW'

# # Convert the mapping to a DataFrame for easier viewing
# mapping_df = pd.DataFrame(list(sr_to_wba_map.items()), columns=['SR_number', 'WBA_name'])

# # Preview result
# print("\n=== SR to WBA Mapping Preview ===")
# print(mapping_df.head())

# Save mapping to CSV if needed
# mapping_df.to_csv("sr_to_wba_mapping.csv", index=False)


In [ ]:
with open(wresl_path, 'r') as f:
    lines = f.readlines()

# Parse mappings
sr_to_wba_map = {}

for line in lines:
    line = line.strip()

    # Handle standard: define indxWBA_2 {value 1 }
    match_wba = re.match(r'define\s+indxWBA_([0-9A-Za-z]+)\s+\{value\s+(\d+)\s+\}', line)
    if match_wba:
        wba_id, sr_num = match_wba.groups()
        sr_key = f"SR{int(sr_num):02d}"       # e.g. 1 → SR01
        wba_value = f"WBA{wba_id}"            # e.g. 2 → WBA2
        sr_to_wba_map[sr_key] = wba_value
        continue

    # Handle special case: define indxDETAW {value 42 }
    match_detaw = re.match(r'define\s+indxDETAW\s+\{value\s+(\d+)\s+\}', line)
    if match_detaw:
        sr_num = match_detaw.group(1)
        sr_key = f"SR{int(sr_num):02d}"       # e.g. 42 → SR42
        sr_to_wba_map[sr_key] = "DETAW"

# Convert to DataFrame
mapping_df = pd.DataFrame(list(sr_to_wba_map.items()), columns=["SR_number", "WBA_name"])
print(mapping_df.head(10))  # check the DETAW row

mapping_df.to_csv("sr_to_wba_mapping.csv", index=False)


In [ ]:

with open(wresl_path, "r") as file:
    lines = file.readlines()

mapping_records = []

i = 1
for line in lines:
    line = line.strip()
    # print("line " + str(i) + ":")
    # print(line)
    match_wba = re.match(r'define\s+indxWBA_([0-9A-Za-z]+)\s+\{value\s+(\d+)\s+\}', line)
    # print("match_wba " + str(i) + ":")
    # print(match_wba)
    
    if match_wba:
        wba_id, sr_num = match_wba.groups()
        # print("wba_id " + str(i) + ":")
        # print(wba_id)
        # print("sr_num " + str(i) + ":")
        # print(sr_num)
        mapping_records.append({
            "Subregion_ID": f"SR{int(sr_num):02d}",
            "WBA_ID": f"WBA{wba_id}"
        })
        continue
    i = i + 1
    match_detaw = re.match(r'define\s+indxDETAW\s+\{value\s+(\d+)\s+\}', line)
    if match_detaw:
        sr_num = match_detaw.group(1)
        mapping_records.append({
            "Subregion_ID": f"SR{int(sr_num):02d}",
            "WBA_ID": "DETAW"
        })

# print("mapping_records:")
# print(mapping_records)

wresl_df = pd.DataFrame(mapping_records).sort_values("Subregion_ID")
display(wresl_df)

wresl_df.to_csv(output_mappingcsv_path, index=False)
print(f"Saved WRESL mapping to: {output_mappingcsv_path}")


## Monthly, Annual Data and Trend 

In [ ]:
# MODIFIED
INPUT_GW_CSV = DataOutPath
STORAGE_FILE = wba_storage_csv_path
WBA_FILE     = wba_csv_path

# Time window (C2VSim range)
WINDOW_START = window_start
WINDOW_END   = window_end
START_YEAR_PRECLIP = start_year

# =======================
# DETAW area handling
# =======================
# If you know the exact DETAW area (in acres), set it here (e.g., 123456.0).
DETAW_AREA_ACRES: Optional[float] = None
# If the above is None, we will use the sum of all WBA GIS_Acres as the area for DETAW conversion.
USE_TOTAL_WBA_ACRES_FOR_DETAW = True

# =======================
# Helpers
# =======================
def load_gw1_df(csv_path: Path) -> pd.DataFrame:
    na_vals = ["", "NA", "NaN", "nan", "-", "--"]
    try:
        df = pd.read_csv(
            csv_path,
            header=[0,1,2,3,4],
            index_col=0,
            parse_dates=True,
            low_memory=False,
            na_values=na_vals
        )
        return df
    except Exception:
        df = pd.read_csv(csv_path, index_col=0, parse_dates=True, na_values=na_vals)
        if len(df.columns) and isinstance(df.columns[0], str) and "|" in df.columns[0]:
            tuples = [tuple(str(c).split("|")) for c in df.columns]
            if all(len(t) == 5 for t in tuples):
                df.columns = pd.MultiIndex.from_tuples(
                    tuples, names=["Model", "VarTag", "Type", "Timestep", "Unit"]
                )
        return df

def normalize_id(s: str) -> str:
    if not isinstance(s, str):
        s = str(s)
    s = s.strip().upper()
    if s.startswith("WBA"):
        s = s[3:]
    return s.zfill(2) if s.isdigit() else s

def normalize_storage_col(col: str) -> Optional[str]:
    if not isinstance(col, str):
        return None
    if col.upper().startswith("WBA"):
        s = col.replace("WBA", "", 1)
        for suffix in ("_STORAGE_AF", "_Storage_AF", "_storage_af"):
            if s.endswith(suffix):
                s = s[:-len(suffix)]
                break
        s = s.strip().upper()
        return s.zfill(2) if s.isdigit() else s
    elif col.upper().startswith("DETAW"):
        return "DETAW"
    return None

def compute_cv_df(df):
    """
    Compute coefficient of variation (std/mean) for each WBA_s#### column,
    and return a DataFrame with scenarios as rows and WBA IDs as columns.
    """
    records = []

    for col in df.columns:
        if "_s" not in col:
            continue
        wba_id, scen_raw = col.split("_s", 1)
        scenario = f"s{scen_raw}"

        series = df[col].dropna()
        if len(series) == 0:
            continue

        mean_val = series.mean()
        std_val = series.std()
        cv = std_val / mean_val if mean_val != 0 else np.nan

        records.append({"scenario": scenario, "WBA": wba_id, "CV": cv})

    cv_df = (
        pd.DataFrame(records)
        .pivot(index="scenario", columns="WBA", values="CV")
        .sort_index(axis=1)
        .sort_index(axis=0)
    )

    return cv_df

# --- Helper: normalize WBA name by stripping leading zeros in the numeric prefix ---
def normalize_wba_name(wba: str) -> str:
    if not isinstance(wba, str):
        return str(wba)
    s = wba.strip().upper()
    # Keep non-WBA names as-is (e.g., DETAW)
    if not s.startswith("WBA"):
        return s
    core = s[3:]
    # Case: digits (with optional trailing letters), e.g. "02", "07N", "17S"
    m = re.match(r'^(\d+)([A-Z].*)?$', core)
    if m:
        num = str(int(m.group(1)))  # drops leading zeros
        tail = m.group(2) or ''
        return "WBA" + num + tail
    # Case: leading zeros before letters (rare), e.g. "0N" -> "N"
    m2 = re.match(r'^0+([A-Z].*)$', core)
    if m2:
        return "WBA" + m2.group(1)
    # Fallback: leave core as-is
    return "WBA" + core

# =======================
# 1) Load inputs
# =======================
gw1_df = load_gw1_df(INPUT_GW_CSV)

wba_df = pd.read_csv(WBA_FILE)
required_cols = {"fid", "GIS_Acres", "WBA_ID"}
missing = required_cols - set(wba_df.columns)
if missing:
    raise ValueError(f"CalSim3_WBA.csv must contain {required_cols}, missing {missing}")

# Maps from WBA table
sr_to_fid    = {f"SR{int(fid):02d}": fid for fid in wba_df["fid"]}
fid_to_acres = dict(zip(wba_df["fid"], wba_df["GIS_Acres"]))

# mapping_df should already exist: ["SR_number", "WBA_name"]
# NOTE: leaving as-is per your original design — this must be defined upstream.
sr_to_wba = dict(zip(mapping_df["SR_number"], mapping_df["WBA_name"]))

# =======================
# 2) Build monthly FT from gw1_df (WBAxx_s00xx, DETAW_s00xx)
# =======================
series_map = {}

for col in gw1_df.columns:
    if not isinstance(col, tuple) or len(col) < 5:
        continue

    model, var_tag, var_type, timestep, unit = col[:5]

    # Handle WBA/SR style (values assumed TAF; convert to FT via area & 1000 AF/TAF)
    if isinstance(var_tag, str) and re.match(r'^SR\d+:TOT_s\d{4}$', var_tag):
        sr, rest = var_tag.split(":")
        scen_raw = rest.split("_s")[-1]
        scenario = f"s{int(scen_raw):04d}"

        if sr not in sr_to_wba:
            continue
        wba_name = sr_to_wba[sr]
        fid = sr_to_fid.get(sr)
        area_acres = fid_to_acres.get(fid)
        if area_acres is None or float(area_acres) == 0.0:
            continue

        series = pd.to_numeric(gw1_df[col], errors="coerce")
        series = series[series.index >= pd.Timestamp(f"{START_YEAR_PRECLIP}-01-01")]
        values_ft = (series / float(area_acres)) * 1000.0  # TAF -> AF -> FT

        neg_idx = np.where(values_ft < 0)[0]
        if len(neg_idx) > 0:
            values_ft = values_ft.iloc[:neg_idx[0]]

        simple_name = f"{wba_name}_{scenario}"
        series_map[simple_name] = values_ft

    # Handle DETAW style (convert AF → FT by dividing by DETAW area)
    elif isinstance(var_tag, str) and re.match(r'^DETAW:TOT_s\d{4}$', var_tag):
        sr, rest = var_tag.split(":")
        scen_raw = rest.split("_s")[-1]
        scenario = f"s{int(scen_raw):04d}"

        series_af = pd.to_numeric(gw1_df[col], errors="coerce")
        series_af = series_af[series_af.index >= pd.Timestamp(f"{START_YEAR_PRECLIP}-01-01")]

        # Determine area to normalize DETAW AF to FT
        if DETAW_AREA_ACRES is not None:
            detaw_area = float(DETAW_AREA_ACRES)
        elif USE_TOTAL_WBA_ACRES_FOR_DETAW:
            detaw_area = float(pd.to_numeric(wba_df["GIS_Acres"], errors="coerce").sum())
        else:
            raise ValueError("DETAW area is not set. Provide DETAW_AREA_ACRES or enable USE_TOTAL_WBA_ACRES_FOR_DETAW.")

        if detaw_area <= 0:
            raise ValueError(f"Invalid DETAW area: {detaw_area}")

        values_ft = series_af / detaw_area  # AF / acre = ft

        simple_name = f"DETAW_{scenario}"
        series_map[simple_name] = values_ft

if series_map:
    monthly_from_tot = pd.concat(series_map, axis=1)
    monthly_from_tot = monthly_from_tot.loc[WINDOW_START:WINDOW_END]
else:
    monthly_from_tot = pd.DataFrame(index=getattr(gw1_df, "index", pd.DatetimeIndex([])))

# =======================
# 3) Build s0000 from storage
# =======================
storage_df = pd.read_csv(
    STORAGE_FILE,
    index_col=0,
    parse_dates=True,
    low_memory=False,
    na_values=["", "NA", "NaN", "nan", "-", "--"]
)

wba_df["WBA_ID_norm"] = wba_df["WBA_ID"].apply(normalize_id)
acres_map = (
    wba_df[["WBA_ID_norm", "GIS_Acres"]]
    .dropna(subset=["GIS_Acres"])
    .drop_duplicates(subset=["WBA_ID_norm"])
    .set_index("WBA_ID_norm")["GIS_Acres"]
    .astype("float64")
    .to_dict()
)

s0000_result = {}
for raw_col in storage_df.columns:
    norm = normalize_storage_col(raw_col)
    if norm is None:
        continue

    af_series = pd.to_numeric(storage_df[raw_col], errors="coerce")

    if norm == "DETAW":
        # Convert DETAW AF → FT using the same area rule as above
        if DETAW_AREA_ACRES is not None:
            detaw_area = float(DETAW_AREA_ACRES)
        elif USE_TOTAL_WBA_ACRES_FOR_DETAW:
            detaw_area = float(pd.to_numeric(wba_df["GIS_Acres"], errors="coerce").sum())
        else:
            raise ValueError("DETAW area is not set for s0000 conversion.")

        if detaw_area <= 0:
            raise ValueError(f"Invalid DETAW area for s0000: {detaw_area}")

        ft_series = af_series / detaw_area  # AF / acre = ft
        s0000_result["DETAW_s0000"] = ft_series
    else:
        if norm not in acres_map:
            continue
        acres = float(acres_map[norm])
        if acres <= 0.0:
            continue
        ft_series = af_series / acres  # AF / acre = ft
        # s0000_result[f"WBA{norm}_s0000"] = ft_series

        wba_name_clean = normalize_wba_name(f"WBA{norm}")
        s0000_result[f"{wba_name_clean}_s0000"] = ft_series

if s0000_result:
    s0000_df = pd.DataFrame(s0000_result, index=storage_df.index)
    s0000_df = s0000_df.loc[WINDOW_START:WINDOW_END]
else:
    s0000_df = pd.DataFrame(index=storage_df.index)

# =======================
# 4) Combine
# =======================
combined_monthly = pd.concat([monthly_from_tot, s0000_df], axis=1).sort_index(axis=1)
combined_monthly = combined_monthly.loc[WINDOW_START:WINDOW_END]


if not combined_monthly.empty:
    combined_annual = combined_monthly.resample("YE").mean()
    combined_annual.index = combined_annual.index.year
else:
    combined_annual = pd.DataFrame(index=[])


# --- Save FT dataframes ---
combined_monthly_path = os.path.join(data_output_dir, monthly_ft_filename)
combined_annual_path = os.path.join(data_output_dir, annual_ft_filename)
combined_monthly.to_csv(combined_monthly_path)
combined_annual.to_csv(combined_annual_path)

print("✓ Saved Feet versions:")
print(f"  Monthly FT: {combined_monthly_path}")
print(combined_monthly.head(5))
print(f"  Annual  FT: {combined_annual_path}")
print(combined_annual.head(5))

# MODIFIED (removed)
# combined_monthly.to_csv(combined_monthly_path)
# combined_annual.to_csv(combined_annual_path)

# =======================
# 4a) Convert to Acre-Feet (AF)
# =======================
af_monthly_map = {}
af_annual_map = {}

for col in combined_monthly.columns:
    if "_s" not in col:
        continue

    # Split into WBA and scenario
    wba_part, scen_part = col.split("_s", 1)
    scenario = f"s{scen_part}"

    # --- Determine area in acres ---
    if wba_part == "DETAW":
        if DETAW_AREA_ACRES is not None:
            area_acres = float(DETAW_AREA_ACRES)
        elif USE_TOTAL_WBA_ACRES_FOR_DETAW:
            area_acres = float(pd.to_numeric(wba_df["GIS_Acres"], errors="coerce").sum())
        else:
            raise ValueError("DETAW area not set for AF conversion.")
    else:
        wba_id = wba_part.replace("WBA", "")
        fid = None
        try:
            fid = int(wba_id)
        except ValueError:
            pass
        area_acres = fid_to_acres.get(fid) or acres_map.get(wba_id)

    if area_acres is None or float(area_acres) <= 0:
        continue

    # --- Convert ft → AF ---
    ft_series = combined_monthly[col]
    af_series = ft_series * float(area_acres)

    af_monthly_map[col] = af_series
    af_annual_map[col] = af_series.resample("YE").sum(min_count=1)

# Build in one concatenation (fast, avoids fragmentation)
combined_af_monthly = pd.concat(af_monthly_map, axis=1)
combined_af_annual = pd.concat(af_annual_map, axis=1)

# MODIFIED (added)
combined_af_annual.index = combined_af_annual.index.year

# --- Save AF dataframes ---
combined_af_monthly_path = os.path.join(data_output_dir, monthly_af_filename)
combined_af_annual_path  = os.path.join(data_output_dir, annual_af_filename)
combined_af_monthly.to_csv(combined_af_monthly_path)
combined_af_annual.to_csv(combined_af_annual_path)

print("✓ Saved Acre-Feet versions:")
print(f"  Monthly AF: {combined_af_monthly_path}")
print(combined_af_monthly.head(5))
print(f"  Annual  AF: {combined_af_annual_path}")
print(combined_af_annual.head(5))

# =======================
# 4b) Express as Percentage of Baseline
# =======================

pct_monthly_map = {}
pct_annual_map = {}

for col in combined_af_monthly.columns:
    if "_s" not in col:
        continue

    # Split into WBA and scenario
    wba_part, scen_part = col.split("_s", 1)
    scenario_num = int(scen_part)
    
    # Determine baseline scenario column
    baseline_scen_num = baseline_scenario  # assumed to be defined elsewhere
    baseline_col = f"{wba_part}_{baseline_scen_num}"
    # print("Baseline column: ", baseline_col)
    
    if baseline_col not in combined_af_monthly.columns:
        # Skip if baseline column doesn't exist
        print("Baseline column does not exist in DF!", baseline_col)
        continue

    # --- Compute percentage relative to baseline ---
    baseline_series = combined_af_monthly[baseline_col]
    pct_series = (combined_af_monthly[col] / baseline_series) * 100

    pct_monthly_map[col] = pct_series
    pct_annual_map[col] = pct_series.resample("YE").mean()  # annual average percent

# Build in one concatenation
combined_volpct_monthly = pd.concat(pct_monthly_map, axis=1)
combined_volpct_annual = pd.concat(pct_annual_map, axis=1)

# MODIFIED (added)
combined_volpct_annual.index = combined_volpct_annual.index.year

# --- Save PERCENT dataframes ---
combined_volper_monthly_path = os.path.join(data_output_dir, monthly_volper_filename)
combined_volper_annual_path  = os.path.join(data_output_dir, annual_volper_filename)
combined_volpct_monthly.to_csv(combined_volper_monthly_path)
combined_volpct_annual.to_csv(combined_volper_annual_path)

print("✓ Saved VOL PER versions:")
print(f"  Monthly VOL PERCENT: {combined_volper_monthly_path}")
print(combined_volpct_monthly.head(5))
print(f"  Annual  VOL PERCENT: {combined_volper_annual_path}")
print(combined_volpct_annual.head(5))

# =======================
# 4c) Compute CV for AF dataframes
# =======================
cv_af_monthly_df = compute_cv_df(combined_af_monthly)
cv_af_annual_df  = compute_cv_df(combined_af_annual)

# --- Save CV dataframes ---
cv_af_monthly_path = os.path.join(data_output_dir, monthly_af_cv_filename)
cv_af_annual_path  = os.path.join(data_output_dir, annual_af_cv_filename)
cv_af_monthly_df.to_csv(cv_af_monthly_path)
cv_af_annual_df.to_csv(cv_af_annual_path)

print("✓ Saved Acre-Feet CV dataframes:")
print(f"  Monthly AF CV: {cv_af_monthly_path}")
print(cv_af_monthly_df.head(5))
print(f"  Annual  AF CV: {cv_af_annual_path}")
print(cv_af_annual_df.head(5))

# =======================
# 4d) Compute CV for FT dataframes
# =======================
cv_monthly_df = compute_cv_df(combined_monthly)
cv_annual_df = compute_cv_df(combined_annual)

# --- Save to CSV ---
cv_monthly_path = os.path.join(data_output_dir, monthly_ft_cv_filename)
cv_annual_path = os.path.join(data_output_dir, annual_ft_cv_filename)

cv_monthly_df.to_csv(cv_monthly_path)
cv_annual_df.to_csv(cv_annual_path)

print("✓ Saved FT CV dataframes:")
print("Monthly FT CV:", cv_monthly_path)
print("Annual FT CV:", cv_annual_path)
display(cv_annual_df.head())

# =======================
# 4e) Compute CV for PER VOL dataframes
# =======================
volpercv_monthly_df = compute_cv_df(combined_volpct_monthly)
volpercv_annual_df  = compute_cv_df(combined_volpct_monthly)

# --- Save CV dataframes ---
volpercv_monthly_path = os.path.join(data_output_dir, monthly_volpercv_filename)
volpercv_annual_path  = os.path.join(data_output_dir, annual_volpercv_filename)
volpercv_monthly_df.to_csv(volpercv_monthly_path)
volpercv_annual_df.to_csv(volpercv_annual_path)

print("✓ Saved VOL PER CV dataframes:")
print(f"  Monthly VOL PER CV: {volpercv_monthly_path}")
display(volpercv_monthly_df.head())
print(f"  Annual  VOL PER CV: {volpercv_annual_path}")
display(volpercv_annual_df.head())

# MODIFIED (the following lines are alread commented, but can be removed entirely)# # =======================
# # 4f) Compute CV for PER LEV dataframes
# # =======================
# levpercv_monthly_df = compute_cv_df(combined_levpct_monthly)
# levpercv_annual_df  = compute_cv_df(combined_levpct_monthly)

# # --- Save CV dataframes ---
# levpercv_monthly_path = os.path.join(data_output_dir, monthly_levpercv_filename)
# levpercv_annual_path  = os.path.join(data_output_dir, annual_levpercv_filename)
# levpercv_monthly_df.to_csv(levpercv_monthly_path)
# levpercv_annual_df.to_csv(levpercv_annual_path)

# print("✓ Saved LEV PER CV dataframes:")
# print(f"  Monthly LEV PER CV: {levpercv_monthly_path}")
# print(f"  Annual  LEV PER CV: {levpercv_annual_path}")
# display(volpercv_annual_df.head())


In [ ]:
# MODIFIED
# --- Time variable in months since start (assumes combined_monthly is monthly) ---
time_numeric = np.arange(len(combined_monthly)).reshape(-1, 1)

# --- Compute slopes (ft/month) and and last 10 years avg - first 10 avg values (ft) for every WBA_s#### column ---
records = []
for col in combined_monthly.columns:
    if "_s" not in col:
        continue
    wba_raw, scen_raw = col.split("_s", 1)  # e.g., "WBA02", "0001"
    scenario = f"s{scen_raw}"
    ts = combined_monthly[col].dropna()
    y = combined_monthly[col].to_numpy(dtype=float)
    mask = ~np.isnan(y)
    
    if mask.sum() > 1:
        # compute trend
        model = LinearRegression().fit(time_numeric[mask], y[mask])
        slope = model.coef_[0]  # ft/month
        wba_norm = normalize_wba_name(wba_raw)
        records.append({"scenario": scenario, "WBA": wba_norm, "slope_ft_per_month": slope})

        # compute n-year start vs end means
        ts = ts.sort_index()  # ensure chronological order
        years = ts.index.year
        first_years = ts[years <= years.min() + YearsToAverage - 1]
        last_years = ts[years >= years.max() - YearsToAverage + 1]
    
        if len(first_years) > 0 and len(last_years) > 0:
            mean_start = first_years.mean()
            mean_end = last_years.mean()
            diff_ft = mean_end - mean_start  # ft
        else:
            diff_ft = np.nan

        # --- Store results ---
        records.append({
            "scenario": scenario,
            "WBA": wba_norm,
            "slope_ft_per_month": slope,
            "mean_start_ft": mean_start,
            "mean_end_ft": mean_end,
            "diff_last_vs_first_ft": diff_ft
        })
        
records_df = pd.DataFrame.from_records(records)

# --- If duplicates appear after normalization (e.g., WBA07N & WBA7N), merge by mean ---
trends_df = (
    records_df
    .groupby(["scenario", "WBA"], as_index=False)["slope_ft_per_month"]
    .mean()
)

diff_df = (
    records_df
    .groupby(["scenario", "WBA"], as_index=False)["diff_last_vs_first_ft"]
    .mean()
)

# --- Pivot: scenarios as rows, WBAs as columns ---
trend_matrix = trends_df.pivot(index="scenario", columns="WBA", values="slope_ft_per_month")
diff_matrix = diff_df.pivot(index="scenario", columns="WBA", values="diff_last_vs_first_ft")

# --- (Optional) tidy sorting: scenarios numerically, WBA columns by numeric then suffix ---
def scen_key(s):  # "s0001" -> 1
    try:
        return int(str(s).lstrip("sS"))
    except Exception:
        return 10**9

def wba_key(w):  # "WBA17S" -> (17, "S"), "DETAW" -> (inf, "DETAW")
    w = str(w).upper()
    if w.startswith("WBA"):
        core = w[3:]
        m = re.match(r'^(\d+)([A-Z].*)?$', core)
        if m:
            return (int(m.group(1)), m.group(2) or "")
    return (10**9, w)

trend_matrix = trend_matrix.reindex(sorted(trend_matrix.index, key=scen_key))
trend_matrix = trend_matrix.reindex(sorted(trend_matrix.columns, key=wba_key), axis=1)
diff_matrix = diff_matrix.reindex(sorted(diff_matrix.index, key=scen_key))
diff_matrix = diff_matrix.reindex(sorted(diff_matrix.columns, key=wba_key), axis=1)

# --- Save CSVs ---
trends_out_path = os.path.join(data_output_dir, trend_filename)
trend_matrix.to_csv(trends_out_path)
diffs_out_path = os.path.join(data_output_dir, diff_filename)
diff_matrix.to_csv(diffs_out_path)

print("✓ Trends file written:", trends_out_path)
print("Shape:", trend_matrix.shape)
print(trend_matrix.head())
print("✓ Diffs file written:", diffs_out_path)
print("Shape:", diff_matrix.shape)
print(diff_matrix.head())


# Plot histograms to find a natural break in the trends of baseline scenario

## Specify number of bins

In [ ]:
nBins = 50

## Plot baseline trend histogram

In [ ]:
# MODIFIED
if SaveHistogram:
    # --- Select the row from trend_matrix ---
    baseline_data = trend_matrix.loc[baseline_scenario].dropna()
    
    # --- Plot histogram ---
    bins = 20  # you can set nBins here
    plt.figure(figsize=(10, 6))
    counts, bin_edges, _ = plt.hist(baseline_data, bins=bins, edgecolor='black')
    
    save_name = f"TrendsHistogram_{baseline_scenario}.png"
    save_path = os.path.join(trends_output_dir, save_name)
    
    # Add more x-axis ticks using bin edges
    plt.xticks(np.round(bin_edges, 4), rotation=45)
    plt.title(f"Histogram of Monthly Trends (ft/month) for {baseline_scenario}")
    plt.xlabel("Slope Value (ft/month)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()
    print(f"✓ Saved: {save_path}")

## Specify clipping quantiles

In [ ]:
lQuant = 0
hQuant = 0.5

## Plot clipped baseline trend histogram

In [ ]:
# MODIFIED
if SaveHistogram:

    # --- Select the row from trend_matrix ---
    baseline_data = trend_matrix.loc[baseline_scenario].dropna()
    
    # --- Quantile thresholds (set these before running) ---
    # Example: lQuant = 0.05; hQuant = 0.95
    lVal, hVal = np.quantile(baseline_data.values, [lQuant, hQuant])
    
    # --- Clip data ---
    clipped_data = baseline_data.values.clip(lVal, hVal)
    
    # --- Plot histogram ---
    bins = 20  # or nBins if you’ve defined it elsewhere
    plt.figure(figsize=(10, 6))
    counts, bin_edges, _ = plt.hist(clipped_data, bins=bins, edgecolor='black')
    
    save_name = f"ClippedTrendsHistogram_{baseline_scenario}.png"
    save_path = os.path.join(trends_output_dir, save_name)
    
    # Add more x-axis ticks using bin edges
    plt.xticks(np.round(bin_edges, 4), rotation=45)
    plt.title(f"Histogram of Monthly Trends (ft/month) for {baseline_scenario} (after clipping)")
    plt.xlabel("Slope Value (ft/month)")
    plt.ylabel("Frequency")
    plt.grid(True)
    plt.savefig(save_path, dpi=300, bbox_inches="tight")
    plt.show()
    plt.close()
    print(f"✓ Saved: {save_path}")

## Notes: Where to put the break? What threshold to use to distinguish moderate from severe? Propose: -0.015; For the GW model (s0000) we tried -0.0025; For s0002, trying -0.0075

In [ ]:
severe_decline_threshold=-0.0075

## Plot trends

In [ ]:
# MODIFIED
if SaveTrendPlots:
    # === Parameters ===
    scenarios_to_plot = sorted({baseline_scenario, *index_names})
    # scenarios_to_plot = {"s0002", "s0015"}
    print("Scenarios to plot:")
    print(scenarios_to_plot)
    drop_threshold = 1000
    start_year = 1960
    
    # === Use combined_monthly directly ===
    # combined_monthly should already be loaded in your workspace
    gw1_df_filtered = combined_monthly.copy()
    
    # === Iterate over columns (WBA_s####) ===
    for col in gw1_df_filtered.columns:
        if "_s" not in col:
            continue
    
        wba_id, scenario = col.split("_")
        if scenario not in scenarios_to_plot:
            continue
    
        ts = gw1_df_filtered[col].dropna()
        ts = ts[ts.index >= pd.Timestamp(f"{start_year}-01-01")]
    
        diffs = ts.diff()
        drop_indices = diffs[diffs < -drop_threshold].index
        if not drop_indices.empty:
            cutoff_idx = drop_indices[0]
            ts = ts[ts.index <= cutoff_idx]
            drop_year = cutoff_idx.year
        else:
            drop_year = 2015  # clipped already
    
        if len(ts) < 2:
            continue  # skip if not enough data
    
        # Fit trendline
        x = (ts.index - ts.index[0]).days / 365.25
        y = ts.values
        slope, intercept = np.polyfit(x, y, 1)
        trend = slope * x + intercept
    
        # === Plot ===
        plt.figure(figsize=(10, 4))
        plt.plot(ts.index, y, label="Observed", marker="o")
        plt.plot(ts.index, trend, linestyle="--", label=f"Trend (slope={slope:.6f})")
        
        # Ensure title not clipped by adding pad
        plt.title(f"{wba_id} under {scenario} (end year: {drop_year})", pad=20)
        plt.xlabel("Date")
        plt.ylabel("Groundwater Storage (FT)")
        plt.grid(True)
        plt.legend()
        plt.tight_layout()
        
        # Save
        save_name = f"{scenario}_{wba_id}.png"
        save_path = os.path.join(trends_output_dir, save_name)
        plt.savefig(save_path, dpi=300, bbox_inches="tight")
        plt.show()
        plt.close()
        print(f"✓ Saved: {save_path}")
    

In [ ]:
print(output_dir)

## Trend Comparison

In [ ]:
# MODIFIED

# === Time axis (months from start to end) ===
time_index = pd.date_range(start="1960-01-01", end="2015-12-31", freq="M")
t_months = np.arange(len(time_index))  # 0,1,2,...

# === Time axis (clipped to match combined_monthly) ===
time_index = pd.date_range(start="1973-10-31", end="2015-09-30", freq="M")
t_months = np.arange(len(time_index))  # 0,1,2,... months


# === Function ===
def plot_trendline_comparison_from_matrix(scenario_code, baseline_code=baseline_scenario, save_dir=None):
    if scenario_code not in trend_matrix.index or baseline_code not in trend_matrix.index:
        print(f"Scenario {scenario_code} or baseline {baseline_code} not in trend_matrix")
        return

    scenario_slopes = trend_matrix.loc[scenario_code]
    baseline_slopes = trend_matrix.loc[baseline_code]

    shared_wbas = sorted(set(scenario_slopes.dropna().index) & set(baseline_slopes.dropna().index))
    if not shared_wbas:
        print(f"No shared WBAs for {scenario_code} and {baseline_code}")
        return

    ncols = 3
    nrows = math.ceil(len(shared_wbas) / ncols)
    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(5 * ncols, 3 * nrows))
    axes = axes.flatten()

    for i, wba in enumerate(shared_wbas):
        slope = scenario_slopes[wba]
        slope_base = baseline_slopes[wba]

        # Build linear trends: start at 0, slope in ft/month
        trend = slope * t_months
        trend_base = slope_base * t_months

        ax = axes[i]
        ax.plot(time_index, trend, color="red", linestyle="--", label=f"{scenario_code} trend")
        ax.plot(time_index, trend_base, color="black", linestyle="--", label=f"{baseline_code} trend")

        ax.set_title(wba)
        ax.set_xlim(time_index[0], time_index[-1])
        ax.set_ylabel("FT (relative)")
        ax.grid(True)

        # Annotate slope values
        ax.text(0.01, 0.95, f"{scenario_code} slope: {slope:.6f}", transform=ax.transAxes,
                fontsize=8, color="red", verticalalignment='top')
        ax.text(0.01, 0.85, f"{baseline_code} slope: {slope_base:.6f}", transform=ax.transAxes,
                fontsize=8, color="black", verticalalignment='top')

        ax.legend(loc="lower right", fontsize=7, frameon=True)

    # Hide unused subplots
    for j in range(i + 1, len(axes)):
        axes[j].axis("off")

    fig.suptitle(f"Trendline Comparison (ft/month): {scenario_code} vs {baseline_code}", fontsize=15)
    fig.tight_layout(rect=[0, 0.04, 1, 0.96])

    if save_dir:
        os.makedirs(save_dir, exist_ok=True)
        save_path = os.path.join(save_dir, f"{scenario_code}_vs_{baseline_code}_trendlines.png")
        plt.savefig(save_path, dpi=300)
        print(f"✓ Saved: {save_path}")
        plt.show()
        plt.close()
    else:
        plt.show()

# === Call for all scenarios except baseline ===
if SaveTrendComparisonsPlots:
    all_scenarios = [sc for sc in trend_matrix.index if sc != baseline_scenario]
    for sc in all_scenarios:
        plot_trendline_comparison_from_matrix(scenario_code=sc, baseline_code=baseline_scenario, save_dir=trends_comparisons_output_dir)


## Compute tiers

In [ ]:
# MODIFIED
def assign_tiers_from_trends(trend_matrix, baseline, output_dir, filename, severe_decline_threshold=-0.015):
    if baseline not in trend_matrix.index:
        raise ValueError(f"Baseline scenario {baseline} not found in trend_matrix")

    tier_matrix = pd.DataFrame(index=trend_matrix.index, columns=trend_matrix.columns)

    for wba_col in trend_matrix.columns:
        baseline_slope = trend_matrix.loc[baseline, wba_col]

        for scenario in trend_matrix.index:
            slope = trend_matrix.loc[scenario, wba_col]

            if pd.isna(slope) or pd.isna(baseline_slope):
                tier = np.nan
            elif scenario == baseline:
                tier = 0  # baseline tier
            elif slope >= 0:
                diff = slope - baseline_slope
                tier = 1 if diff >= 0 else 2
            elif slope >= severe_decline_threshold:
                tier = 3
            else:
                tier = 4

            tier_matrix.loc[scenario, wba_col] = tier

    # === Save results ===
    out_path = os.path.join(output_dir, filename)
    tier_matrix.to_csv(out_path)
    print("✓ Tier assignment saved to:", out_path)
    print(tier_matrix.head())

    return tier_matrix

# Call the function
tier_matrix = assign_tiers_from_trends(
    trend_matrix,
    baseline=baseline_scenario,
    output_dir=data_output_dir,
    filename=tier_filename,
    severe_decline_threshold=severe_decline_threshold
)

## Tier maps

In [ ]:
# MODIFIED
if SaveTierMaps:
    wba_shp = gpd.read_file(shapefile_path)
    wba_shp["WBA_ID"] = wba_shp["WBA_ID"].str.strip()
    
    # === Load new tier assignment CSV ===
    tier_df = pd.read_csv(tier_output_path, index_col=0)
    
    # === Fix WBA column names to match shapefile format ===
    new_columns = {}
    for col in tier_df.columns:
        if col.startswith("WBA"):
            suffix = col[3:]
            if suffix.isdigit():
                new_columns[col] = suffix.zfill(2)
            else:
                digits = ''.join(filter(str.isdigit, suffix)).zfill(2)
                letter = ''.join(filter(str.isalpha, suffix))
                new_columns[col] = digits + letter
    tier_df.rename(columns=new_columns, inplace=True)
    
    # === Define atlas-style muted tier colors ===
    tier_colors = {
        1: "#B5CDA3",  # olive green
        2: "#8FBBD9",  # soft dusty blue
        3: "#E6C27A",  # warm khaki
        4: "#D97B6D",  # soft brick red (worst)
    }
    
    # === Vertical shifts for selected WBA labels ===
    label_shifts = {
        "17STOT": 0.015,
        "71TOT": 0.015,
        "22TOT": 0.015,
        "50TOT": -0.015,
        "21TOT": -0.015,
        "12TOT": -0.015,
    }
    
    # === Plot and save maps for each scenario (skip baseline s0000) ===
    for scenario in tier_df.index:
        if scenario == baseline_scenario:  # skip baseline
            continue
    
        # Map tier data to shapefile
        tier_series = tier_df.loc[scenario]
        tier_map = wba_shp.copy()
        tier_map["GroundwaterTier"] = tier_map["WBA_ID"].map(tier_series.to_dict())
    
        # Plot base
        fig, ax = plt.subplots(figsize=(8, 10))
        for tier_val, color in tier_colors.items():
            subset = tier_map[tier_map["GroundwaterTier"] == tier_val]
            if not subset.empty:
                subset.plot(
                    ax=ax,
                    color=color,
                    edgecolor='black',
                    linewidth=0.3,
                    label=f"Tier {tier_val}"
                )
    
        # === Add WBA_ID labels ===
        for idx, row in tier_map.iterrows():
            if pd.notna(row["GroundwaterTier"]):
                x, y = row.geometry.centroid.x, row.geometry.centroid.y
                wba_id = row["WBA_ID"]
                shift = label_shifts.get(wba_id, 0)
                ax.text(
                    x, y + shift, wba_id,
                    fontsize=7, weight='bold', ha='center'
                )
    
        ax.set_title(f"Groundwater Tiers for Scenario {scenario}", fontweight="bold")
        ax.axis("off")
    
        # Add legend
        legend_handles = [mpatches.Patch(color=color, label=f"Tier {tier}")
                          for tier, color in tier_colors.items()]
        ax.legend(handles=legend_handles, title="Tier", loc="lower left", frameon=True)
    
        # Save and show
        save_path = os.path.join(tiers_output_dir, f"GroundWaterTiers_{scenario}.png")
        plt.tight_layout()
        plt.savefig(save_path, dpi=300)
        plt.show()
    
        print(f"✓ Saved map for {scenario} to: {output_dir}")




## Slope rank map

In [ ]:
# MODIFIED
# --- Helper to pad WBA IDs ---
def pad_index(idx):
    import re
    if isinstance(idx, str):
        m = re.match(r'^(\d{1,2})([A-Z]*)$', idx)
        if m:
            num = int(m.group(1))
            suffix = m.group(2)
            return f"{num:02d}{suffix}"
    return idx

if SaveSlopeMap:
    # === Load shapefile ===
    wba_shp = gpd.read_file(shapefile_path)
    wba_shp["WBA_ID"] = wba_shp["WBA_ID"].str.strip().str.upper()
    
    # === Load new slopes CSV (trend_matrix output) ===
    trend_csv = trends_out_path
    
    df = pd.read_csv(trend_csv, index_col=0)
    
    # === Scenarios to plot ===
    scenarios_to_plot = [baseline_scenario]
    
    for scenario in scenarios_to_plot:
        if scenario not in df.index:
            print(f"⚠ Scenario {scenario} not found in {trend_csv}")
            continue
    
        baseline_data = df.loc[scenario].dropna()
    
        # --- Clean WBA names ---
        baseline_data.index = (
            baseline_data.index
            .str.replace("WBA", "", regex=False)
            .str.replace(":TOT", "", regex=False)
            .str.replace("TOT", "", regex=False)
            .str.lstrip("0")
            .str.strip()
            .str.upper()
        )
        baseline_data.index = baseline_data.index.map(pad_index)
    
        # === Join slope data to shapefile ===
        slope_rank = baseline_data.rank().astype(int)
        slope_map = wba_shp.copy()
        slope_map["Slope"] = slope_map["WBA_ID"].map(baseline_data.to_dict())
        slope_map["SlopeRank"] = slope_map["WBA_ID"].map(slope_rank.to_dict())
    
        # === Color setup ===
        num_ranks = slope_rank.nunique()
        cmap = plt.colormaps.get_cmap("coolwarm_r").resampled(num_ranks)
        norm = mcolors.Normalize(vmin=1, vmax=num_ranks)
    
        # === Plot map ===
        fig, ax = plt.subplots(figsize=(8, 10))
        slope_map.plot(
            column="SlopeRank",
            cmap=cmap,
            linewidth=0.3,
            edgecolor='black',
            ax=ax,
            legend=False,
            norm=norm
        )
    
        # Add labels
        for idx, row in slope_map.iterrows():
            if pd.notna(row["SlopeRank"]):
                x, y = row.geometry.centroid.x, row.geometry.centroid.y
                ax.text(x, y, row["WBA_ID"], fontsize=7, weight='bold', ha='center')
    
        # Add colorbar with slope values
        sm = plt.cm.ScalarMappable(cmap=cmap, norm=norm)
        sm._A = []
        rank_to_slope = slope_map.dropna(subset=["Slope", "SlopeRank"]) \
                                 .groupby("SlopeRank")["Slope"].mean().sort_index()
        tick_locs = list(rank_to_slope.index)
        tick_labels = [f"{s:.6f}" for s in rank_to_slope.values]
    
        cbar = fig.colorbar(sm, ax=ax, orientation="vertical", ticks=tick_locs)
        cbar.set_label("Slope Value (ft/month)")
        cbar.ax.yaxis.set_major_locator(FixedLocator(tick_locs))
        cbar.ax.yaxis.set_major_formatter(FixedFormatter(tick_labels))
    
        ax.set_title(f"Slope Rank Map for Scenario {scenario}", fontweight="bold", pad=15)
        ax.axis("off")
        plt.tight_layout()
    
        # === Save ===
        save_path = os.path.join(tiers_output_dir, f"SlopeRankMap_{scenario}.png")
        plt.savefig(save_path, dpi=300)
        plt.show()
    
        print(f"✓ Saved slope rank map for {scenario} to {output_dir}")



In [ ]:
print("Done!")